In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving my_dataset.json to my_dataset.json


In [ ]:
model_path = "/content/drive/MyDrive/TinyLlama-1.1B-Chat-v1.0"
# dataset_path = "/content/drive/MyDrive/my_dataset.json"

In [ ]:
import json
from datasets import Dataset

dataset_path = "/content/drive/MyDrive/pseudo_labeled.jsonl"  # Change if needed

# Load raw JSON
# with open(dataset_path, "r") as f:
    # raw_data = json.load(f)

# # below is for reading jsonl file
raw_data = []
with open(dataset_path, "r", encoding="utf-8") as f:
  for line in f:
      raw_data.append(json.loads(line.strip()))

# Create Hugging Face dataset
dataset = Dataset.from_list(raw_data)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)


In [ ]:
# def format_prompt(example):
#     return f"### Instruction:\n{example['instruction']}\n### Input:\n{example['input']}\n### Response:\n{example['output']}"

def preprocess(example):
#     # prompt = format_prompt(example)
    encoded = tokenizer(example["input"], truncation=True, max_length=512, padding="max_length")
    encoded["labels"] = encoded["input_ids"].copy()
    return encoded

tokenized_dataset = dataset.map(preprocess)
# tokenized_dataset = raw_data(preprocess)

## as my pseudo_jsonl is already in format (instruction,input,output)
# dataset = Dataset.from_list(raw_data)



Map:   0%|          | 0/72 [00:00<?, ? examples/s]

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Typical for llama models
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, config)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./tinyllama-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=7,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True  # If A100 is not available, set this False
    # evaluation_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


<ipython-input-13-92a01ec21fcc>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: patelparthjayantilal93 (patelparthjayantilal93-tata-consultancy-services) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,2.393900
20,2.426000
30,2.432800
40,2.231900
50,2.245500
60,2.265800
70,2.165400
80,2.231200
90,2.099500
100,2.115600


TrainOutput(global_step=126, training_loss=2.232340093642946, metrics={'train_runtime': 173.9838, 'train_samples_per_second': 2.897, 'train_steps_per_second': 0.724, 'total_flos': 1603467101601792.0, 'train_loss': 2.232340093642946, 'epoch': 7.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/tinyllama-finetuned1")
tokenizer.save_pretrained("/content/drive/MyDrive/tinyllama-finetuned1")
# Model training ends here beloq was agent creation part save model by analyizing above training loss so model doesent overfit on data

('/content/drive/MyDrive/tinyllama-finetuned1/tokenizer_config.json',
 '/content/drive/MyDrive/tinyllama-finetuned1/special_tokens_map.json',
 '/content/drive/MyDrive/tinyllama-finetuned1/chat_template.jinja',
 '/content/drive/MyDrive/tinyllama-finetuned1/tokenizer.model',
 '/content/drive/MyDrive/tinyllama-finetuned1/added_tokens.json',
 '/content/drive/MyDrive/tinyllama-finetuned1/tokenizer.json')

In [ ]:

# trained_model_path = "/content/drive/MyDrive/tinyllama-finetuned"


In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import PeftModel
# import torch

# # Load base and LoRA models
# base_model_path = "/content/drive/MyDrive/TinyLlama-1.1B-Chat-v1.0"
# lora_model_path = "/content/drive/MyDrive/tinyllama-finetuned1"

# tokenizer = AutoTokenizer.from_pretrained(base_model_path)
# base_model = AutoModelForCausalLM.from_pretrained(base_model_path)
# model = PeftModel.from_pretrained(base_model, lora_model_path)

# # Move model to GPU
# model = model.to("cuda")
# model.eval()




PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_feat

In [ ]:
# # Define your prompt
# # prompt = """### Instruction:

# # ### Input:
# # What is EMI in terms of real estate?.
# # ### Response:
# # """

# # defining a new prompt so that my model undrstand the AI agent format
# prompt = """You are an agent that uses tools. Use this format:

# Question: What is the contact for user 123?
# Thought: I should use the GetContactDetails tool
# Action: GetContactDetails
# Action Input: "123"
# Observation: Contact sent to frontend
# Final Answer: I have sent the contact info.

# """

# # Tokenize input and move to GPU
# inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# # Generate output
# with torch.no_grad():
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=100,
#         temperature=0.7,
#         top_p=0.9,
#         do_sample=True,
#         eos_token_id=tokenizer.eos_token_id
#     )

# # Decode and print response
# response = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(response.split("### Response:")[-1].strip())

You are an agent that uses tools. Use this format:
 
Question: What is the contact for user 123?
Thought: I should use the GetContactDetails tool
Action: GetContactDetails
Action Input: "123"
Observation: Contact sent to frontend
Final Answer: I have sent the contact info.

Example 2:
You are a user that uses tools. Use this format:

Question: What is the contact for user 123?
Thought: I should use the GetContactDetails tool
Action: GetContactDetails
Action Input: "123"
Observation: Contact sent to frontend
Final Answer: I have sent the contact info.


In [ ]:

#####################################################
# From HERE onwards We are creating AI agent that will call any function as per user request automatically.
# if user tell send me contact details => call getContacts function

SyntaxError: invalid syntax (<ipython-input-1-dd5840ca652d>, line 2)

In [ ]:
# !pip install transformers sentencepiece accelerate langchain

In [ ]:
# from transformers import pipeline

# generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
# from langchain.llms.base import LLM
# from typing import Optional, List

# class TinyLlamaLLM(LLM):
#   def __init__(self,pipeline):
#     super().__init__()
#     self._pipeline=pipeline

#   def _call(self, prompt: str, stop: Optional[List[str]] = None)-> str:
#     output = self._pipeline(prompt, max_new_tokens=256, do_sample=True)
#     return output[0]["generated_text"]

#   @property
#   def _llm_type(self) -> str:
#     return "TinyLlamaLLM"

# llm = TinyLlamaLLM(pipeline=generator)



In [ ]:
# from langchain.agents import Tool

# def get_contact_details(user_id: str)-> str:
#   return f"Returning contact details for user {user_id}"

# def buy_property(property_id: str)-> str:
#   return f"Returning property details for property {property_id}"

# tools = [
#     Tool(
#         name="GetContactDetails",
#         func=get_contact_details,
#         description="get contact info by user ID"),
#     Tool(
#         name="BuyProperty",
#         func=buy_property,
#         description="Buy property by ID")
#     ]

In [ ]:
# from langchain.agents import initialize_agent,AgentType

# agent = initialize_agent(tools=tools, llm=llm, agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)

<ipython-input-10-2d757adf346f>:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools=tools, llm=llm, agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)


In [ ]:
# response = agent.run("please get contact details for user 123")
# print(response)

<ipython-input-11-65f0e46f8e53>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("please get contact details for user 123")




> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

GetContactDetails(user_id: str) -> str - get contact info by user ID
BuyProperty(property_id: str) -> str - Buy property by ID

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [GetContactDetails, BuyProperty]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: please get contact details for user 123
Thought: user 123 has an email address
Action: get contact details for user 123
Action Input: user 123's email address
Observation: get contact details for user 123


Token indices sequence length is longer than the specified maximum sequence length for this model (2488 > 2048). Running this sequence through the model will result in indexing errors
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

GetContactDetails(user_id: str) -> str - get contact info by user ID
BuyProperty(property_id: str) -> str - Buy property by ID

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [GetContactDetails, BuyProperty]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: please get contact details for user 123
Thought:Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

GetContactDetails(user_id: str) ->

KeyboardInterrupt: 